# Analysis of Astrocytes in the medial Arcuate Nucleus Data Set

## Table of contents:

* <a href=#Load>Load Packages and Set Global Variables</a>
    * <a href=#Imports>Imports and Settings</a>
    * <a href=#Global>Global Variables</a> 
* <a href=#Dataloading>Loading Data, Quality Control and Preprocessing</a>
    * <a href=#Counts>Gene numbers and counts with and without mitochondrial RNA</a>
* <a href=#Allcells>All cells - normalization, projection and clustering</a>
* <a href=#Define>Define Cell Types</a>
* <a href=#astrocytes>Astrocytes Only</a>
    * <a href=#Embedding>Embeddings and Clustering</a>
    * <a href=#adipmarkers>Astrocyte Marker Analysis</a>
    * <a href=#topde>Top ranking DE Genes</a>
    * <a href=#count_dist>Count distribution for Gfap, Aldh1l1 and Slc1a3</a>
* <a href=#traject>Gfap and Aldh1l1 only</a>

<a id="Load"></a>

# Load Packages and Set Global Variables

<a id="imports"></a>

## Imports and Settings

In [1]:
import numpy as np
import scanpy as sc
import scipy as sci
import scipy.sparse
import pandas as pd
import seaborn as sb
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colors
from gprofiler import GProfiler
import custom_functions as cf
import matplotlib_venn
import sys
import re
import os

import batchglm
import diffxpy.api as de

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2
sc.settings.verbosity = 3 # amount of output

base_dir = '/Users/viktorian.miok/Documents/consultation/Luiza/single_cell/data/scanpy_AnnData/'
dir_out = '/Users/viktorian.miok/Documents/consultation/Luiza/single_cell/results/'
dir_tables = dir_out+'tables/'
sc_settings_figdir = dir_out+'figures/'
sc_settings_writedir = dir_out+'anndata/'
sc.logging.print_versions()
os.chdir(dir_out)
sc.settings.set_figure_params(dpi=80, scanpy=True)
print (sys.version)

/Users/viktorian.miok/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


-----
anndata     0.7.5
scanpy      1.7.1
sinfo       0.3.1
-----
OpenSSL                             19.1.0
PIL                                 7.0.0
absl                                NA
anndata                             0.7.5
appnope                             0.1.0
astunparse                          1.6.3
autoreload                          NA
backcall                            0.1.0
batchglm                            v0.7.4
cairo                               1.18.2
certifi                             2020.06.20
cffi                                1.13.2
chardet                             3.0.4
cloudpickle                         1.3.0
colorama                            0.4.3
cryptography                        2.8
custom_functions                    NA
cycler                              0.10.0
cython_runtime                      NA
d0500e42e7b9c00c4a62953977ddea7b    NA
dask                                2.18.1
dateutil                            2.8.1
decorator       

In [2]:
#Define a nice colour map for gene expression
colors2 = plt.cm.Reds(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

## Global Variables

All embeddings and clusterings can be saved and loaded into this script. Be carful with overwriting cluster caches as soon as cell type annotation has started as cluster labels may be shuffled.

Set whether anndata objects are recomputed or loaded from cache.

In [3]:
bool_recomp = False

Set whether clustering is recomputed or loaded from saved .obs file. Loading makes sense if the clustering changes due to a change in scanpy or one of its dependencies and the number of clusters or the cluster labels change accordingly.

In [4]:
bool_recluster = False

Set whether cluster cache is overwritten. Note that the cache exists for reproducibility of clustering, see above.

In [5]:
bool_write_cluster_cache = False

Set whether to produce plots, set to False for test runs.

In [6]:
bool_plot = False

Set whether observations should be calculated. If false, it is necessary to read cacheed file that contains the necssary information. It then shows the the distributions of counts and genes, as well as mt_frac after filtering. 
Set to true in order to see the data before filtering and follow the decisions for cutoffs.

In [7]:
bool_create_observations = True

<a id="Dataloading"></a>

# Loading Data, Quality Control and Preprocessing

Read the data in:

In [8]:
if bool_recomp:
    adata_raw1 = sc.read(base_dir + 'MUC26030/filtered_feature_bc_matrix.h5ad')
    adata_raw2 = sc.read(base_dir + 'MUC26031/filtered_feature_bc_matrix.h5ad')
    adata_raw3 = sc.read(base_dir + 'MUC26032/filtered_feature_bc_matrix.h5ad')
    adata_raw = adata_raw1.concatenate([adata_raw2, adata_raw3],
                                       batch_key='diet', 
                                       batch_categories=['chow', 'hfd_5', 'hfd_15']
    )
    sc.write(sc_settings_writedir+'adata_raw.h5ad', adata_raw)
else:
    adata_raw = sc.read(sc_settings_writedir+'adata_raw.h5ad')

<a id="QC"></a>

Summary of steps performed here: Only cells with at least 500 UMIs are kept. Counts per cell are cell library depth normalized. The gene (feature) space is reduced with PCA to 50 PCs. A nearest neighbour graph and umap are computed based on the PC space. Cell are clustered with louvain clustering based on the nearest neighbour graph. Graph abstraction is computed based on the louvain clustering.

In [9]:
sc.pp.filter_cells(adata_raw, min_counts = 1)

The data contains 21143 observations with 31253 different genes. Due to dropouts, some of the observations might not show any counts and genes. In order to calculate the fraction of mitochondrial RNA in the next steps, each observations without counts must be filtered out to prevent NaN from emerging. 

In [10]:
print('Number of cells: {:d}'.format(adata_raw.n_obs))
print('Number of genes: {:d}'.format(adata_raw.shape[1]))
print('Number of cells per diet:')
adata_raw.obs['diet'].value_counts().sort_index()

Number of cells: 21143
Number of genes: 31253
Number of cells per diet:


chow      7116
hfd_5     6204
hfd_15    7823
Name: diet, dtype: int64

### Gene numbers and counts with and without mitochondrial RNA

Create necessary obs:

In [11]:
adata_qc = adata_raw.copy()
adata_qc.obs['n_genes'] = (adata_qc.X > 0).sum(1)
mt_gene_mask = [gene.startswith('mt-') for gene in adata_qc.var_names]
temp_mt_sum = adata_qc[:,mt_gene_mask].X.sum(1)
temp_mt_sum = np.squeeze(np.asarray(temp_mt_sum))
adata_qc.obs['n_counts'] = adata_qc.X.sum(1)
temp_n_counts = adata_qc.obs['n_counts']
adata_qc.obs['mt_frac'] = temp_mt_sum/adata_qc.obs['n_counts']

Plot n_counts and mt_frac:

In [12]:
if bool_plot == True:
    t1 = sc.pl.violin(adata_qc, ['n_counts', 'n_genes', 'mt_frac'], size =1, log=False, jitter=3, multi_panel=True)

In [13]:
if bool_plot==True:
    sc.pl.highest_expr_genes(adata_qc, n_top=20) 

Overall, the data contains a lot of observations with high fractions of mitochondrial RNA. Additionally, most observations show counts below 100, suggesting poor data quality. To further investigate the distributions counts over genes per observations, scatterplots are created:

### Number of Genes versus Number of Counts

In [14]:
if bool_plot == True:
    p1 = sc.pl.scatter(adata_qc, 'n_counts','n_genes',color='mt_frac', size = 5)
    p2 = sc.pl.scatter(adata_qc[adata_qc.obs['n_counts']<5000],'n_counts','n_genes',color = 'mt_frac', size = 5)

### Distribution of Counts and Genes

For the remaining observations, the fraction of mitochondrial RNA is generally very low and at most 20%

In [15]:
if bool_plot == True:
    p6 = sb.distplot(adata_qc.obs['n_counts'], kde = False)
    plt.show()
    p7 = sb.distplot(adata_qc.obs['n_counts'][adata_qc.obs['n_counts']<1000], kde = False)
    plt.show()

In [16]:
if bool_plot == True:
    p9 = sb.distplot(adata_qc.obs['n_genes'],kde = False, bins=60)
    plt.show()
    p10 = sb.distplot(adata_qc.obs['n_genes'][adata_qc.obs['n_genes']<500],kde = False, bins=60)
    plt.show()

### Filtering

In [17]:
# Filter cells according to identified QC thresholds:
print('Total number of cells: {:d}'.format(adata_qc.n_obs))

sc.pp.filter_cells(adata_qc, min_counts = 200)
print('Number of cells after min count filter: {:d}'.format(adata_qc.n_obs))

sc.pp.filter_cells(adata_qc, max_counts = 100000)
print('Number of cells after max count filter: {:d}'.format(adata_qc.n_obs))

adata_qc = adata_qc[adata_qc.obs['mt_frac'] < 0.5]
print('Number of cells after MT filter: {:d}'.format(adata_qc.n_obs))

sc.pp.filter_cells(adata_qc, min_genes = 350)
print('Number of cells after gene filter: {:d}'.format(adata_qc.n_obs))

Total number of cells: 21143


filtered out 35 cells that have more than 100000 counts


Number of cells after min count filter: 21143
Number of cells after max count filter: 21108
Number of cells after MT filter: 20491


filtered out 496 cells that have less than 350 genes expressed
Trying to set attribute `.obs` of view, copying.


Number of cells after gene filter: 19995


In [18]:
#Filter genes:
print('Total number of genes: {:d}'.format(adata_qc.n_vars))

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata_qc, min_cells=20)
print('Number of genes after cell filter: {:d}'.format(adata_qc.n_vars))

Total number of genes: 31253


filtered out 13164 genes that are detected in less than 20 cells


Number of genes after cell filter: 18089


In [19]:
if bool_plot == True:
    p1 = sc.pl.scatter(adata_qc, 'n_counts','n_genes',color='mt_frac', size = 5)
    p3 = sc.pl.scatter(adata_qc[adata_qc.obs['n_counts']<5000],'n_counts','n_genes',color = 'mt_frac', size = 5)

## All cells - normalization, projection and clustering

In [20]:
if bool_recomp == True:
        
    adata_proc = adata_qc.copy()
    adata_proc.raw = adata_proc
    sc.pp.normalize_per_cell(adata_proc)
    sc.pp.log1p(adata_proc)
    sc.pp.combat(adata_proc, key='diet')
    sc.pp.highly_variable_genes(adata_proc, flavor='cell_ranger',n_top_genes=4000)
    sc.pl.highly_variable_genes(adata_proc)
    #adata_proc.X = adata_proc.X.toarray()
    
    sc.pp.pca(adata_proc, n_comps=50, random_state=0, use_highly_variable=True, svd_solver='arpack')
    sc.pp.neighbors(adata_proc, n_neighbors=100, knn=True, method='umap', n_pcs=50, random_state=0)
    sc.tl.umap(adata_proc)
    if bool_recluster == True:
        sc.tl.louvain(adata_proc, resolution=0.5, flavor='vtraag', random_state=0)
        pd.DataFrame(adata_proc.obs).to_csv(path_or_buf =sc_settings_writedir+"obs_adata_proc.csv")
    else:
        obs = pd.read_csv(sc_settings_writedir+'obs_adata_proc.csv')
        adata_proc.obs['louvain']=pd.Series(obs['louvain'].values, dtype = 'category')
    sc.write(sc_settings_writedir+'adata_proc.h5ad',adata_proc)
else:
    adata_proc = sc.read(sc_settings_writedir+'adata_proc.h5ad') 
sc.tl.paga(adata_proc)

running PAGA
    finished: added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns) (0:00:01)


Produce some summarizing plots that show the global characteristics of the data.

In [21]:
if bool_plot == True:
    cf.plot_umap_marker(adata_proc, ['louvain'], save="_all_cells_louvain.svg", use_raw=False)

In [22]:
if bool_plot == True:
    cf.plot_umap_marker(adata_proc, ['louvain'], save="_all_cells_louvain_ondata.svg", use_raw=False, legend_loc='on data')

In [23]:
if bool_plot == True:
    cf.plot_umap_marker(adata_proc, ['n_genes','n_counts','mt_frac'], color_map=mymap, size=10, save="_all_cells_n_gene_count_mt.svg", use_raw = False)

A high fraction of mitochondrial RNA is in cluster 3 and around the central cluster 8.

In [24]:
if bool_plot == True:
    cf.plot_umap_marker(adata_proc, ['Gfap','Aldh1l1'], color_map=mymap, size=20, save="_all_cells_gfap-aldh.svg", use_raw = False)

In [25]:
if bool_plot == True:
    cf.plot_umap_marker(adata_proc, ['diet'], save="_all_cells_diet.svg", use_raw=False)

In [26]:
if bool_plot == True:
    sc.pl.paga(adata_proc, save="_all_cells.svg")

In [27]:
if bool_plot == True:
    cf.cell_percent(adata_proc, cluster='louvain', condition='diet', xlabel='clusters', ylabel='percentage', 
                 title='barplot_all_cells_diet_per_clusters', save=sc_settings_figdir, table=False)

In [28]:
if bool_plot == True:
    aldh_pos = adata_proc.obs_names[np.asarray(adata_proc[:,'Aldh1l1'].X).flatten()>0]
    gfap_pos = adata_proc.obs_names[np.asarray(adata_proc[:,'Gfap'].X).flatten()>0]
    glast_pos = adata_proc.obs_names[np.asarray(adata_proc[:,'Slc1a3'].X).flatten()>0]

    matplotlib_venn.venn3([
        set(aldh_pos),
        set(gfap_pos),
        set(glast_pos)
    ], set_labels = ("Aldh1l1", "Gfap", "Slc1a3"))
    plt.savefig(sc_settings_figdir+'venndiagram_all_cells_gfap-aldh-glast.svg')

In [29]:
if bool_plot == True:
    aldh_pos = adata_proc.obs_names[np.asarray(adata_proc[:,'Aldh1l1'].X).flatten()>0]
    gfap_pos = adata_proc.obs_names[np.asarray(adata_proc[:,'Gfap'].X).flatten()>0]

    matplotlib_venn.venn2([
        set(aldh_pos),
        set(gfap_pos)
    ], set_labels = ("Aldh1l1", "Gfap"))#, "Slc1a3"))
    plt.savefig(sc_settings_figdir+'venndiagram_all_cells_gfap-aldh.svg')

Number of cells in each cluster:

In [30]:
adata_proc.obs["louvain"].value_counts()

0     3912
1     2568
2     2381
3     2263
4     1512
5     1457
6     1346
7     1304
8     1106
9      960
10     618
11     378
12     190
Name: louvain, dtype: int64

# Define Cell Types

<a id="DE"></a>

## DE Genes

In [31]:
sc.tl.rank_genes_groups(adata_proc, groupby='louvain', key_added='rank_genes')

if bool_plot==True:
    sc.pl.rank_genes_groups(adata_proc, key='rank_genes', groups=['0','1','2'], save="_all_cells_1.svg")
    sc.pl.rank_genes_groups(adata_proc, key='rank_genes', groups=['3','4','5'], save="_all_cells_2.svg")
    sc.pl.rank_genes_groups(adata_proc, key='rank_genes', groups=['6','7','8'], save="_all_cells_3.svg")
    sc.pl.rank_genes_groups(adata_proc, key='rank_genes', groups=['9','10','11'], save="_all_cells_4.svg")
    sc.pl.rank_genes_groups(adata_proc, key='rank_genes', groups=['12'], save="_all_cells_5.svg")

ranking genes
    finished: added to `.uns['rank_genes']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:03)


## Define Marker Sets

Define marker sets for some of the expected cell types and add DE genes.

In [32]:
# Leukocyte markers:
astrocyte_markers = ['Slc1a2', 'Slc1a3', 'Aqp4', 'S100b', 'Sox9','Gfap','Aldh1l1','Gja1','Gjb6','Agt','Atp1b2']
microglia_markers = ['Itgam','Tmem119','Cx3cr1','Aif1','Csf1r','P2ry12']
macrophage_markers = ['Mrc1']
oligodendrocyte_markers = ['Olig1','Mog','Mag','Cd9','P11p']
ng2_markers = ['Cspg4','Pdgfra']
neuron_markers = ['Rbfox3','Syp', 'Map2', 'Tubb3','Snap25','Syt1','Th','Gal','Pnoc']
endothelial_markers = ['Cldn5', 'Pecam1','Slco1c1']
mural_markers = ['Mustn1','Pdgfrb','Des']
neural_stem_markers = ['Sox2','Nes','Notch1']
radial_glia_markers = ['Pax6','Hes1']
ependymal_markers = ['Nfx','Foxj1','Cd24a','Vim','Sox2','Ccdc153','Rarres2','Fgf10','Rax','Lhx2']
vlmc_markers = ['Lum','Col1a1','Col3a1']
pars_tuberalis_markers = ['Tshb','Cck']

Only keep markers occurring in data set.

In [33]:
astrocyte_markers = np.array([x for x in astrocyte_markers if x in adata_proc.var_names])
microglia_markers = np.array([x for x in microglia_markers if x in adata_proc.var_names])
macrophage_markers = np.array([x for x in macrophage_markers if x in adata_proc.var_names])
oligodendrocyte_markers = np.array([x for x in oligodendrocyte_markers if x in adata_proc.var_names])
ng2_markers = np.array([x for x in ng2_markers if x in adata_proc.var_names])
neuron_markers = np.array([x for x in neuron_markers if x in adata_proc.var_names])
endothelial_markers = np.array([x for x in endothelial_markers if x in adata_proc.var_names])
mural_markers = np.array([x for x in mural_markers if x in adata_proc.var_names])
neural_stem_markers = np.array([x for x in neural_stem_markers if x in adata_proc.var_names])
radial_glia_markers = np.array([x for x in radial_glia_markers if x in adata_proc.var_names])
ependymal_markers = np.array([x for x in ependymal_markers if x in adata_proc.var_names])
vlmc_markers = np.array([x for x in vlmc_markers if x in adata_proc.var_names])
pars_tuberalis_markers = np.array([x for x in pars_tuberalis_markers if x in adata_proc.var_names])

## Astrocyte Markers

In [34]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, astrocyte_markers.tolist(), save="_all_cells_astrocyte_markers")

## Microglia Markers

In [35]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, microglia_markers.tolist(), save="_all_cells_microglia_markers")

Cluster 1,9 and 10 show expression for the leukocyte marker Ptprc. Tcf25 found in many clusters.

## Macrophage Markers

In [36]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, macrophage_markers.tolist(), save="_all_cells_macrophage_markers")

## Oligodendrocyte Markers

In [37]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, oligodendrocyte_markers.tolist(), save="_all_cells_oligodendrocyte_markers")

The haemoglobin markers and Alas2 show high counts in cluster 5 indicating that cluster 5 corresponds to erythrocytes.

## NG2 Markers

In [38]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, ng2_markers.tolist(), save="_all_cells_ng2_markers")

<a id="Preadipocyte"></a>

## Neuron Markers

In [39]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, neuron_markers.tolist(), save="_all_cells_neuron_markers")

<a id="Natural"></a>

## Endothelial Markers

In [40]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, endothelial_markers.tolist(), save="_all_cells_endothelial_markers")

NK cell markers are expressed in cluster 6, which is inline with their transcriptomic similarity to T cells which are also found in cluster 6.

## Mural Markers

In [41]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, mural_markers.tolist(), save="_all_cells_mural_markers")

## Neural Stem Cells Markers

In [42]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, neural_stem_markers.tolist(), save="_all_cells_neural_stem_markers")

## Radial Glia Cells Markers

In [43]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, radial_glia_markers.tolist(), save="_all_cells_radial_glia_markers")

## Ependymal  Cells Markers

In [44]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, ependymal_markers.tolist(), save="_all_cells_ependymal_markers")

## Vascular and leptomeningeal cells markers

In [45]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, vlmc_markers.tolist(), save="_all_cells_vlmc_markers")

## Pars Tuberalis markers

In [46]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, pars_tuberalis_markers.tolist(), save="_all_cells_pars_tuberalis_markers")

## Vascular and leptomeningeal cells markers

In [47]:
if bool_plot == True:
    cf.plot_violin_marker(adata_proc, vlmc_markers.tolist(), save="_all_cells_vlmc_markers")

## Summary heatmap, dotplot and stacked_violin for cluster assignments

In [48]:
selected_astrocyte_markers = ['Slc1a2', 'Slc1a3', 'Aqp4', 'S100b', 'Sox9','Gfap','Aldh1l1','Gja1','Gjb6','Agt','Atp1b2']
selected_microglia_markers = ['Itgam','Tmem119','Cx3cr1','Csf1r','Aif1','P2ry12']
selected_macrophage_markers = ['Mrc1']
selected_oligodendrocyte_markers = ['Olig1','Mog','Mag','Cd9','P11p']
selected_ng2_markers = ['Cspg4','Pdgfra']
selected_neuron_markers = ['Rbfox3','Syp', 'Map2', 'Tubb3','Snap25','Syt1','Th','Gal','Pnoc']
selected_endothelial_markers = ['Cldn5', 'Pecam1','Slco1c1']
selected_mural_markers = ['Mustn1','Pdgfrb','Des']
selected_neural_stem_markers = ['Sox2','Nes','Notch1']
selected_radial_glia_markers = ['Pax6','Hes1']
selected_ependymal_markers = ['Nfx','Foxj1','Cd24a','Vim','Sox2','Ccdc153','Rarres2','Fgf10','Rax','Lhx2']
selected_vlmc_markers = ['Lum','Col1a1','Col3a1']


In [49]:
selected_cell_markers = selected_astrocyte_markers + \
selected_microglia_markers + \
selected_macrophage_markers + \
selected_oligodendrocyte_markers + \
selected_ng2_markers + \
selected_neuron_markers + \
selected_endothelial_markers + \
selected_mural_markers + \
selected_neural_stem_markers + \
selected_radial_glia_markers + \
selected_ependymal_markers + \
selected_vlmc_markers

In [50]:
marker_genes_dict = {'Astrocytes': ['Slc1a2', 'Slc1a3', 'Aqp4', 'S100b', 'Sox9','Gfap','Aldh1l1','Gja1','Gjb6','Agt','Atp1b2'],
                     'Microglia': ['Itgam','Tmem119','Cx3cr1','Aif1','Csf1r','P2ry12'],
                     'Macrophage': ['Mrc1'],
                     'Oligodendrocyte': ['Olig1','Mog','Mag','Cd9'],
                     'ng2': ['Cspg4','Pdgfra'],
                     'Neuron': ['Rbfox3','Syp', 'Map2', 'Tubb3','Snap25','Syt1','Th','Gal','Pnoc'],
                     'Endothel': ['Cldn5', 'Pecam1','Slco1c1'],
                     'Mural': ['Mustn1','Pdgfrb','Des'],
                     'Neural_stem': ['Sox2','Nes','Notch1'],
                     'Radial': ['Pax6','Hes1'],
                     'Ependymal': ['Foxj1','Cd24a','Vim','Sox2','Ccdc153','Rarres2','Fgf10','Rax','Lhx2'],
                     'vlmc': ['Lum','Col1a1','Col3a1']}

In [51]:
if bool_plot==True:
    sc.pl.heatmap(
        adata=adata_proc, 
        var_names=marker_genes_dict, 
        groupby="louvain", 
        use_raw=False, 
        log=False, 
        dendrogram=True, 
        var_group_rotation=90, 
        show_gene_labels=True, 
        show=True, 
        save="_all_cells_celltypes_markers.svg"
    )

In [52]:
if bool_plot==True:
    sc.pl.dotplot(
        adata=adata_proc,
        var_names=marker_genes_dict, 
        groupby='louvain',
        use_raw=False, 
        log=False, 
        dendrogram=True, 
        var_group_rotation=90, 
        show=True, 
        save="all_cells_celltypes_markers.svg")

In [53]:
if bool_plot==True:
    sc.pl.stacked_violin(adata_proc, 
                         marker_genes_dict, 
                         groupby='louvain', 
                         show=True,
                         use_raw=False,
                         dendrogram=True,
                         cmap='viridis_r',
                         save="all_cells_celltypes_markers.svg")

In [54]:
if bool_plot==True:
    plt.figure(figsize=(7,7))
    cell_annotation = sc.tl.marker_gene_overlap(adata_proc, marker_genes_dict, key='rank_genes', 
                                                normalize='data')
    sb.heatmap(cell_annotation, cbar=False, annot=True)
    plt.savefig(sc_settings_figdir+'heatmap_all_cells_rank_genes_cell_annotation.svg')

## Gfap vs. Aldh1l1 - per diet

###  chow diet cells

In [55]:
if bool_plot == True:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4), gridspec_kw={'wspace':0.8})
    ax1_dict = ax1.scatter(adata_proc[adata_proc.obs['diet']=='chow', ][:, ['Gfap']].X, 
                           adata_proc[adata_proc.obs['diet']=='chow', ][:, ['Aldh1l1']].X, 
                           s=9, cmap='seismic', c='dodgerblue')
    ax1.set_title('chow')
    ax1.set_ylabel('Gfap')
    ax1.set_xlabel('Aldh1l1')
    ax2_dict = sc.pl.umap(adata_proc[adata_proc.obs['diet']=='chow', ], color=['louvain'], size=5, ax=ax2, show=False)
    plt.savefig(sc_settings_figdir+'umap_all_cells_chow_gfap-aldh_1')

In [56]:
if bool_plot == True:
    cf.plot_umap_marker(adata_proc[adata_proc.obs['diet']=='chow', ], ['Gfap','Aldh1l1'],
                        color_map=mymap, size=20, save="_all_cells_chow_gfap-aldh.svg", use_raw = False)

### hfd_5 diet cells

In [57]:
if bool_plot == True:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4), gridspec_kw={'wspace':0.8})
    ax1_dict = ax1.scatter(adata_proc[adata_proc.obs['diet']=='hfd_5', ][:, ['Gfap']].X, 
                           adata_proc[adata_proc.obs['diet']=='hfd_5', ][:, ['Aldh1l1']].X, 
                           s=9, cmap='seismic', c='darkorange')
    ax1.set_title('chow')
    ax1.set_ylabel('Gfap')
    ax1.set_xlabel('Aldh1l1')
    ax2_dict = sc.pl.umap(adata_proc[adata_proc.obs['diet']=='hfd_5', ], color=['louvain'], size=5, ax=ax2, show=False)
    plt.savefig(sc_settings_figdir+'umap_all_cells_hfd5_gfap-aldh_1.svg')

In [58]:
if bool_plot == True:
    cf.plot_umap_marker(adata_proc[adata_proc.obs['diet']=='hfd_5', ], ['Gfap','Aldh1l1'],
                        color_map=mymap, size=20, save="_all_cells_hfd5_gfap-aldh.svg", use_raw = False)

### hfd15 diet cells

In [59]:
if bool_plot == True:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4), gridspec_kw={'wspace':0.8})
    ax1_dict = ax1.scatter(adata_proc[adata_proc.obs['diet']=='hfd_15', ][:, ['Gfap']].X, 
                           adata_proc[adata_proc.obs['diet']=='hfd_15', ][:, ['Aldh1l1']].X, 
                           s=9, cmap='seismic', c='green')
    ax1.set_title('chow')
    ax1.set_ylabel('Gfap')
    ax1.set_xlabel('Aldh1l1')
    ax2_dict = sc.pl.umap(adata_proc[adata_proc.obs['diet']=='hfd_15', ], color=['louvain'], size=5, ax=ax2, show=False)
    plt.savefig(sc_settings_figdir+'umap_all_cells_hfd15_gfap-aldh_1.svg')

In [60]:
if bool_plot == True:
    cf.plot_umap_marker(adata_proc[adata_proc.obs['diet']=='hfd_15', ], ['Gfap','Aldh1l1'],
                        color_map=mymap, size=20, save="_all_cells_hfd15_gfap-aldh.svg", use_raw = False)

## UMAP with assigned cell types

In [61]:
new_cluster_names = {
    '0': "astrocytes",
    '1': "ependymal",
    '2': "ependymal",
    '3': "neurons",
    '4': "ng2",
    '5': "endothelial",
    '6': "oligodendrocytes",
    '7': "microglia",
    '8': "neurons",
    '9': "microglia",
    '10': "mural",
    '11': "oligodendrocytes",
    '12': "oligodendrocytes"
}
adata_proc.obs['celltypes'] = [new_cluster_names[x] for x in  adata_proc.obs['louvain']]

In [62]:
if bool_plot == True:
    cf.plot_umap_marker(adata_proc, ['celltypes'], save="_all_cells_celltypes.svg", use_raw=False)

In [63]:
if bool_plot == True:
    cf.plot_umap_marker(adata_proc, ['celltypes'], save="_all_cells_celltypes_ondata.svg", use_raw=False, legend_loc='on data')

In [64]:
if bool_plot == True:
    sc.pl.paga(adata_proc, save="_all_cells_celltypes.svg")

# Astorcytes only

<a id="Embedding"></a>

## Embedding and Clustering

In [65]:
if bool_recomp == True:  
    cell_ids_astro = np.asarray(adata_proc.obs_names)[
        [x in ['astrocytes'] 
         for x in np.asarray(adata_proc.obs['celltypes'].values)]
    ]
    adata_astro = adata_raw[cell_ids_astro,:].copy()
    adata_astro.obs['n_genes'] = (adata_astro.X > 0).sum(1)
    adata_astro.obs['n_counts'] = adata_astro.X.sum(1)
    mt_gene_mask = [gene.startswith('mt-') for gene in adata_astro.var_names]
    temp_mt_sum = adata_astro[:,mt_gene_mask].X.sum(1)
    temp_mt_sum = np.squeeze(np.asarray(temp_mt_sum))
    temp_n_counts = adata_astro.obs['n_counts']
    adata_astro.obs['mt_frac'] = temp_mt_sum/adata_astro.obs['n_counts']
    adata_astro.raw = adata_astro
    sc.pp.normalize_per_cell(adata_astro)
    sc.pp.log1p(adata_astro)
    sc.pp.highly_variable_genes(adata_astro,n_top_genes=4000)
    sc.pl.highly_variable_genes(adata_astro)
    adata_astro.X = adata_astro.X.toarray()
    
    sc.pp.pca(adata_astro, n_comps=50, use_highly_variable = True, random_state=0, svd_solver='arpack')
    sc.pp.neighbors(adata_astro, n_neighbors=100, knn=True, method='umap', n_pcs=50, random_state=0)
    sc.tl.umap(adata_astro)
    if bool_recluster == True:
        sc.tl.louvain(adata_astro, resolution=1, flavor='vtraag', random_state=0)
        pd.DataFrame(adata_astro.obs).to_csv(path_or_buf=sc_settings_writedir+'obs_adata_astro.csv')
    else:
        obs = pd.read_csv(sc_settings_writedir+'obs_adata_astro.csv')
        adata_astro.obs['louvain']=pd.Series(obs['louvain'].values, dtype = 'category')
    sc.write(sc_settings_writedir+'adata_astro.h5ad',adata_astro)
else:
    adata_astro = sc.read(sc_settings_writedir+'adata_astro.h5ad') 
sc.tl.paga(adata_astro)

running PAGA
    finished: added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns) (0:00:00)


In [66]:
if bool_plot == True:
    cf.plot_umap_marker(adata_astro, ['louvain'], save="_astrocyte_louvain.svg", use_raw=False, size=30)

In [67]:
if bool_plot == True:
    cf.plot_umap_marker(adata_astro, ['louvain'], save="_astrocyte_louvain_ondata.svg", use_raw=False, legend_loc='on data', size=30)

In [68]:
if bool_plot == True:
    cf.plot_umap_marker(adata_astro, ['n_genes','n_counts','mt_frac'], color_map=mymap, size=30, save="_astrocyte_n_gene_count_mt.svg", use_raw = False)

A high fraction of mitochondrial RNA is in cluster 3 and around the central cluster 8.

In [69]:
if bool_plot == True:
    cf.plot_umap_marker(adata_astro, ['Gfap','Aldh1l1'], color_map=mymap, size=30, save="_astrocyte_markers.svg", use_raw = False)

In [70]:
if bool_plot == True:
    cf.plot_umap_marker(adata_astro, ['diet'], save="_astrocytes_diet.svg", size=30, use_raw=False)

In [71]:
if bool_plot == True:
    sc.pl.paga(adata_astro, save="_astrocytes.svg")

In [72]:
if bool_plot == True:
    cf.cell_percent(adata_astro, cluster='louvain', condition='diet', xlabel='clusters', ylabel='percentage', 
                 title='barplot_astrocytes_diet_per_clusters', save=sc_settings_figdir, table=False)

In [73]:
if bool_plot == True:
    aldh_pos = adata_astro.obs_names[np.asarray(adata_astro[:,'Aldh1l1'].X).flatten()>0]
    gfap_pos = adata_astro.obs_names[np.asarray(adata_astro[:,'Gfap'].X).flatten()>0]
    glast_pos = adata_astro.obs_names[np.asarray(adata_astro[:,'Slc1a3'].X).flatten()>0]

    matplotlib_venn.venn3([
        set(aldh_pos),
        set(gfap_pos),
        set(glast_pos)
    ], set_labels = ("Aldh1l1", "Gfap", "Slc1a3"))
    plt.savefig(sc_settings_figdir+'venndiagram_astrocytes_gfap-aldh-glast.svg')

In [74]:
if bool_plot == True:
    aldh_pos = adata_astro.obs_names[np.asarray(adata_astro[:,'Aldh1l1'].X).flatten()>0]
    gfap_pos = adata_astro.obs_names[np.asarray(adata_astro[:,'Gfap'].X).flatten()>0]

    matplotlib_venn.venn2([
        set(aldh_pos),
        set(gfap_pos)
    ], set_labels = ("Aldh1l1", "Gfap"), set_colors = ('lime','magenta'))
    plt.savefig(sc_settings_figdir+'venndiagram_astrocytes_gfap-aldh.svg')

### Count distribution for Aldh1l1 and Gfap

#### Single and double positive counts¶

Define booleans for single and double positive counts of Gfap and Aldh1l1

In [75]:
non_boolean_int = np.array((adata_astro[:,'Gfap'].X<=0) & (adata_astro[:,'Aldh1l1'].X<=0), dtype=int)

gfap_single_boolean = (adata_astro[:,'Gfap'].X>0) & (adata_astro[:,'Aldh1l1'].X<=0)
aldh_single_boolean = (adata_astro[:,'Aldh1l1'].X>0) & (adata_astro[:,'Gfap'].X<=0) 
single_boolean_int = np.array((gfap_single_boolean | aldh_single_boolean), dtype=int)*1

gfap_aldh_double_boolean = (adata_astro[:,'Gfap'].X>0) & (adata_astro[:,'Aldh1l1'].X>0)
double_boolean_int = np.array((gfap_aldh_double_boolean), dtype=int)*2

non_boolean_int *=0

In [76]:
gfap_single_pos = adata_astro[:,'Gfap'].X[gfap_single_boolean]
aldh_single_pos = adata_astro[:,'Aldh1l1'].X[aldh_single_boolean]

print('Gfap Single Positive ',len(gfap_single_pos))
print('Aldh1l1 Single Positive ',len(aldh_single_pos))

gfap_aldh_double_pos = adata_astro[:,'Gfap'].X[gfap_aldh_double_boolean]

print('Gfap/Aldh1l1 Double Positive ',len(gfap_aldh_double_pos))


Gfap Single Positive  682
Aldh1l1 Single Positive  850
Gfap/Aldh1l1 Double Positive  625


<a id="adipmarkers"></a>

In [77]:
adata_astro.obs['sdt_pos'] = np.array((non_boolean_int + single_boolean_int + double_boolean_int), dtype=str)
adata_astro.obs['s_pos'] = np.array((np.array(gfap_single_boolean, dtype=int)*1)+(np.array(aldh_single_boolean, dtype=int)*2), dtype=str)
adata_astro.obs['d_pos'] = np.array((np.array(gfap_aldh_double_boolean, dtype=int)*1), dtype=str)

# make them categorical
adata_astro.obs['sdt_pos'] = pd.Series(adata_astro.obs['sdt_pos'], dtype="category")
adata_astro.obs['s_pos'] = pd.Series(adata_astro.obs['s_pos'], dtype="category")
adata_astro.obs['d_pos'] = pd.Series(adata_astro.obs['d_pos'], dtype="category")

In [78]:
if bool_plot == True:
    new_cluster_names = ['Non Positives','Single Positives','Double Positives']
    adata_astro.rename_categories('sdt_pos', new_cluster_names)
    sc.pl.umap(adata_astro, color=['sdt_pos'], size=30, palette=["lightgrey","tomato","blue"], 
               save="_astrocytes_gfap-aldh_single_double_positiv.svg")
    new_cluster_names = ['Non Single Positives','Gfap Single Positives','Aldh1l1 Single Positives']
    adata_astro.rename_categories('s_pos', new_cluster_names)
    sc.pl.umap(adata_astro[adata_astro.obs['s_pos']!='Non Single Positives'], color=['s_pos'],
               size=30, palette=["magenta","lime"], save="_astrocytes_gfap-aldh_single_positiv.svg")
    new_cluster_names = ['Non Double Positives','Gfap/Aldh1l1 Double Positive']
    adata_astro.rename_categories('d_pos', new_cluster_names)
    sc.pl.umap(adata_astro[adata_astro.obs['d_pos']!='Non Double Positives'], color=['d_pos'], size=30,
              save="_astrocytes_gfap-aldh_doble_positiv.svg")

### Astrocyte chow diet

In [79]:
if bool_plot == True:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4), gridspec_kw={'wspace':0.8})
    ax1_dict = ax1.scatter(adata_astro[adata_astro.obs['diet']=='chow', ][:, ['Gfap']].X, 
                           adata_astro[adata_astro.obs['diet']=='chow', ][:, ['Aldh1l1']].X, 
                           s=9, cmap='seismic', c='dodgerblue')
    ax1.set_title('chow')
    ax1.set_ylabel('Gfap')
    ax1.set_xlabel('Aldh1l1')
    ax2_dict = sc.pl.umap(adata_astro[adata_astro.obs['diet']=='chow', ], color=['louvain'], size=30, ax=ax2, show=False)
    plt.savefig(sc_settings_figdir+'gfap-aldh_astrocytes_chow.svg')

In [80]:
if bool_plot == True:
    cf.plot_umap_marker(adata_astro, ['Gfap', 'Aldh1l1'], save="_astrocyte_chow_gfap-aldh.svg", size=30, 
                        color_map=mymap, use_raw=False)

In [81]:
if bool_plot == True:
    aldh_pos = adata_astro[adata_astro.obs['diet']=='chow', ].obs_names[np.asarray(adata_astro[adata_astro.obs['diet']=='chow', ][:,'Aldh1l1'].X).flatten()>0]
    gfap_pos = adata_astro[adata_astro.obs['diet']=='chow', ].obs_names[np.asarray(adata_astro[adata_astro.obs['diet']=='chow', ][:,'Gfap'].X).flatten()>0]

    matplotlib_venn.venn2([
        set(aldh_pos),
        set(gfap_pos)
    ], set_labels = ("Aldh1l1", "Gfap"), set_colors = ('lime','magenta'))
    plt.savefig(sc_settings_figdir+'venn_diagram_astrocytes_chow_gfap-aldh.svg')

### Astrocyte hfd_5 diet

In [82]:
if bool_plot == True:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4), gridspec_kw={'wspace':0.8})
    ax1_dict = ax1.scatter(adata_astro[adata_astro.obs['diet']=='hfd_5', ][:, ['Gfap']].X, 
                           adata_astro[adata_astro.obs['diet']=='hfd_5', ][:, ['Aldh1l1']].X, 
                           s=9, cmap='seismic', c='darkorange')
    ax1.set_title('hfd_5')
    ax1.set_ylabel('Gfap')
    ax1.set_xlabel('Aldh1l1')
    ax2_dict = sc.pl.umap(adata_astro[adata_astro.obs['diet']=='hfd_5', ], color=['louvain'], size=30, ax=ax2, show=False)
    plt.savefig(sc_settings_figdir+'gfap-aldh_astrocytes_hfd5.svg')

In [83]:
if bool_plot == True:
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='hfd_5', ], ['Gfap', 'Aldh1l1'],
                        save="_astrocyte_hfd5_gfap-aldh.svg", size=30, color_map=mymap, use_raw=False)

In [84]:
if bool_plot == True:
    aldh_pos = adata_astro[adata_astro.obs['diet']=='hfd_5', ].obs_names[np.asarray(adata_astro[adata_astro.obs['diet']=='hfd_5', ][:,'Aldh1l1'].X).flatten()>0]
    gfap_pos = adata_astro[adata_astro.obs['diet']=='hfd_5', ].obs_names[np.asarray(adata_astro[adata_astro.obs['diet']=='hfd_5', ][:,'Gfap'].X).flatten()>0]

    matplotlib_venn.venn2([
        set(aldh_pos),
        set(gfap_pos)
    ], set_labels = ("Aldh1l1", "Gfap"), set_colors = ('lime','magenta'))
    plt.savefig(sc_settings_figdir+'venn_diagram_astrocytes_hfd5_gfap-aldh.svg')

### Astrocyte hfd_15 diet

In [85]:
if bool_plot == True:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4), gridspec_kw={'wspace':0.8})
    ax1_dict = ax1.scatter(adata_astro[adata_astro.obs['diet']=='hfd_15', ][:, ['Gfap']].X, 
                           adata_astro[adata_astro.obs['diet']=='hfd_15', ][:, ['Aldh1l1']].X, 
                           s=9, cmap='seismic', c='green')
    ax1.set_title('hfd_15')
    ax1.set_ylabel('Gfap')
    ax1.set_xlabel('Aldh1l1')
    ax2_dict = sc.pl.umap(adata_astro[adata_astro.obs['diet']=='hfd_15', ], color=['louvain'], size=30, ax=ax2, show=False)
    plt.savefig(sc_settings_figdir+'gfap-aldh_astrocytes_hfd15.svg')

In [86]:
if bool_plot == True:
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='hfd_15', ], ['Gfap', 'Aldh1l1'],
                        save="_astrocyte_hfd15_gfap-aldh.svg", size=30, color_map=mymap, use_raw=False)

In [87]:
if bool_plot == True:
    aldh_pos = adata_astro[adata_astro.obs['diet']=='hfd_15', ].obs_names[np.asarray(adata_astro[adata_astro.obs['diet']=='hfd_15', ][:,'Aldh1l1'].X).flatten()>0]
    gfap_pos = adata_astro[adata_astro.obs['diet']=='hfd_15', ].obs_names[np.asarray(adata_astro[adata_astro.obs['diet']=='hfd_15', ][:,'Gfap'].X).flatten()>0]

    matplotlib_venn.venn2([
        set(aldh_pos),
        set(gfap_pos)
    ], set_labels = ("Aldh1l1", "Gfap"), set_colors = ('lime','magenta')) 
    plt.savefig(sc_settings_figdir+'venn_diagram_astrocytes_hfd15_gfap-aldh.svg')

## Cells expression Gfap and Aldh1l1

In [88]:
adata_astro.obs['gfap_aldh'] = np.select([((adata_astro[:,'Gfap'].X>0) & (adata_astro[:,'Aldh1l1'].X==0)), 
                                          ((adata_astro[:,'Gfap'].X==0) & (adata_astro[:,'Aldh1l1'].X>0)),
                                          ((adata_astro[:,'Gfap'].X>0) & (adata_astro[:,'Aldh1l1'].X>0)), 
                                          ((adata_astro[:,'Gfap'].X==0) & (adata_astro[:,'Aldh1l1'].X==0))],
                                          ['gfap_only','aldh_only','both','none'])

### Chow

In [89]:
if bool_plot==True:
    data = adata_astro[adata_astro.obs['diet']=='chow', ].obs['gfap_aldh'].value_counts().sort_index()
    def func(pct, allvals):
        absolute = int(pct/100.*np.sum(allvals))
        return "{:.1f}%\n({:d})".format(pct, absolute)

    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(16, 6), subplot_kw=dict(aspect="equal"))
    ax0 = sc.pl.umap(adata_astro[adata_astro.obs['diet']=='chow', ], 
                     color=['gfap_aldh'], size=30, ax=ax0, show=False, palette=['green','blue','magenta','gainsboro'])
    wedges, texts, autotexts = ax1.pie(data, autopct=lambda pct: func(pct, data), textprops=dict(color="w"), 
                                       colors=['green','blue','magenta','gainsboro'])
    ax1.set_title('Chow diet cells - markers share')

    fig.savefig(sc_settings_figdir+'umap_pie_chart_astrocyte_chow_gfap-aldh.svg')

### HFD_5

In [90]:
if bool_plot==True:
    data = adata_astro[adata_astro.obs['diet']=='hfd_5', ].obs['gfap_aldh'].value_counts().sort_index()
    def func(pct, allvals):
        absolute = int(pct/100.*np.sum(allvals))
        return "{:.1f}%\n({:d})".format(pct, absolute)

    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(16, 6), subplot_kw=dict(aspect="equal"))
    ax0 = sc.pl.umap(adata_astro[adata_astro.obs['diet']=='hfd_5', ], 
                     color=['gfap_aldh'], size=30, ax=ax0, show=False)
    wedges, texts, autotexts = ax1.pie(data, autopct=lambda pct: func(pct, data), textprops=dict(color="w"))
    ax1.set_title('Chow diet cells - markers share')

    fig.savefig(sc_settings_figdir+'umap_pie_chart_astrocyte_hfd5_gfap-aldh.svg')

### HFD_15

In [91]:
if bool_plot==True:   
    data = adata_astro[adata_astro.obs['diet']=='hfd_15', ].obs['gfap_aldh'].value_counts().sort_index()
    def func(pct, allvals):
        absolute = int(pct/100.*np.sum(allvals))
        return "{:.1f}%\n({:d})".format(pct, absolute)

    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(16, 6), subplot_kw=dict(aspect="equal"))
    ax0 = sc.pl.umap(adata_astro[adata_astro.obs['diet']=='hfd_15', ], 
                     color=['gfap_aldh'], size=30, ax=ax0, show=False)
    wedges, texts, autotexts = ax1.pie(data, autopct=lambda pct: func(pct, data), textprops=dict(color="w"))
    ax1.set_title('Chow diet cells - markers share')

    fig.savefig(sc_settings_figdir+'umap_pie_chart_astrocyte_hfd15_gfap-aldh.svg')

In [92]:
if bool_plot == True:
    cf.cell_percent(adata_astro, cluster='diet', condition='gfap_aldh', xlabel='clusters', ylabel='percentage', 
                 title='barplot_per_diet_astrocytes_gfap-aldh', save=sc_settings_figdir, table=False)

## Per marker and diet

In [93]:
adata_astro.obs['gfap_only'] = adata_astro.obs['gfap_aldh']=='gfap_only'
adata_astro.obs['aldh_only'] = adata_astro.obs['gfap_aldh']=='aldh_only'
adata_astro.obs['both'] = adata_astro.obs['gfap_aldh']=='both'

In [94]:
# make them categorical
adata_astro.obs['gfap_only'] = pd.Series(adata_astro.obs['gfap_only'], dtype="category")
adata_astro.obs['aldh_only'] = pd.Series(adata_astro.obs['aldh_only'], dtype="category")
adata_astro.obs['both'] = pd.Series(adata_astro.obs['both'], dtype="category")

In [95]:
adata_astro.rename_categories('gfap_only', ['none','gfap_only'])

In [96]:
adata_astro.rename_categories('aldh_only', ['none','aldh_only'])
adata_astro.rename_categories('both', ['none','both'])

### chow

In [97]:
if bool_plot==True:
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='chow', ], ['gfap_only'], palette=['gainsboro','magenta'],
                        save="_astrocytes_gfap_only_chow.svg", size=30, color_map=mymap, use_raw=False)

In [98]:
if bool_plot==True:  
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='chow', ], ['aldh_only'], palette=['gainsboro','green'],
                        save="_astrocytes_aldh_only_chow.svg", size=30, color_map=mymap, use_raw=False)

In [99]:
if bool_plot==True:
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='chow', ], ['both'], palette=['gainsboro','blue'],
                        save="_astrocytes_both_chow.svg", size=30, color_map=mymap, use_raw=False)

In [100]:
if bool_plot==True:
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='chow', ], ['gfap_aldh'], 
                        palette=['green','blue','magenta','gainsboro'],
                        save="_astrocytes_gfap_aldh_chow.svg", size=30, color_map=mymap, use_raw=False)

### Differential gene expression - chow

In [101]:
astro_0 = adata_astro[:,~adata_astro.var.index.isin(['Gfap','Aldh1l1'])]

In [102]:
if bool_plot==True:
    astro0 = astro_0[(astro_0.obs['diet']=='chow') & 
                    (astro_0.obs['gfap_aldh'].isin(['aldh_only','gfap_only'])), ]
    dets_mark = de.test.t_test(data=astro0, sample_description=astro0.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    dets_mark_summary
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05,corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50, 
                           log2_fc_threshold=3.7, save=sc_settings_figdir+"dge_volcano_chow_aldh-gfap",
                           suffix="_astrocytes.svg")

In [103]:
if bool_plot==True:
    astro0 = astro_0[(astro_0.obs['diet']=='chow') & 
                    (astro_0.obs['gfap_aldh'].isin(['both','gfap_only'])), ]
    dets_mark = de.test.t_test(data=astro0, sample_description=astro0.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05,corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50,
                           log2_fc_threshold=3.5, save=sc_settings_figdir+"dge_volcano_chow_both-gfap",
                           suffix="_astrocytes.svg")

In [104]:
if bool_plot==True:
    astro0 = astro_0[(astro_0.obs['diet']=='chow') & 
                    (astro_0.obs['gfap_aldh'].isin(['aldh_only','both'])), ]
    dets_mark = de.test.t_test(data=astro0, sample_description=astro0.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50, 
                           log2_fc_threshold=3.5, save=sc_settings_figdir+"dge_volcano_chow_aldh-both",
                           suffix="_astrocytes.svg")

### hfd 5

In [105]:
if bool_plot==True:
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='hfd_5', ], ['gfap_only'], palette=['gainsboro','magenta'],
                        save="_astrocytes_gfap_only_hfd5.svg", size=30, color_map=mymap, use_raw=False)

In [106]:
if bool_plot==True:
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='hfd_5', ], ['aldh_only'], palette=['gainsboro','green'],
                        save="_astrocytes_aldh_only_hfd5.svg", size=30, color_map=mymap, use_raw=False)

In [107]:
if bool_plot==True: 
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='hfd_5', ], ['both'], palette=['gainsboro','blue'],
                        save="_astrocytes_both_hfd5.svg", size=30, color_map=mymap, use_raw=False)    

In [108]:
if bool_plot==True:
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='hfd_5', ], ['gfap_aldh'], 
                        palette=['green','blue','magenta','gainsboro'],
                        save="_astrocytes_gfap_aldh_hfd5.svg", size=30, color_map=mymap, use_raw=False)

### Differential gene expression - hfd_5

In [109]:
astro_5 = adata_astro[:,~adata_astro.var.index.isin(['Gfap','Aldh1l1'])]

In [110]:
if bool_plot==True:
    astro5 = astro_5[(astro_5.obs['diet']=='hfd_5') & 
                    (astro_5.obs['gfap_aldh'].isin(['aldh_only','gfap_only'])), ]
    dets_mark = de.test.t_test(data=astro5, sample_description=astro5.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    dets_mark_summary
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50, 
                           log2_fc_threshold=3.5, save=sc_settings_figdir+"dge_volcano_hfd5_aldh-gfap",
                           suffix="_astrocytes.svg")

In [111]:
if bool_plot==True:
    astro5 = astro_5[(astro_5.obs['diet']=='hfd_5') & 
                    (astro_5.obs['gfap_aldh'].isin(['both','gfap_only'])), ]
    dets_mark = de.test.t_test(data=astro5, sample_description=astro5.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50,
                           log2_fc_threshold=3.5, save=sc_settings_figdir+"dge_volcano_hfd5_both-gfap",
                           suffix="_astrocytes.svg")

In [112]:
if bool_plot==True:
    astro5 = astro_5[(astro_5.obs['diet']=='hfd_5') & 
                    (astro_5.obs['gfap_aldh'].isin(['aldh_only','both'])), ]
    dets_mark = de.test.t_test(data=astro5, sample_description=astro5.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50, 
                           log2_fc_threshold=3.5, save=sc_settings_figdir+"dge_volcano_hfd5_aldh-both",
                           suffix="_astrocytes.svg")

### hfd 15

In [113]:
if bool_plot==True:
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='hfd_15', ], ['gfap_only'], palette=['gainsboro','magenta'],
                        save="_astrocytes_gfap_only_hfd15.svg", size=30, color_map=mymap, use_raw=False)

In [114]:
if bool_plot==True:
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='hfd_15', ], ['aldh_only'], palette=['gainsboro','green'],
                        save="_astrocytes_aldh_only_hfd15.svg", size=30, color_map=mymap, use_raw=False)

In [115]:
if bool_plot==True:
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='hfd_15', ], ['both'], palette=['gainsboro','blue'],
                        save="_astrocytes_both_hfd15.svg", size=30, color_map=mymap, use_raw=False)

In [116]:
if bool_plot==True:
    cf.plot_umap_marker(adata_astro[adata_astro.obs['diet']=='hfd_15', ], ['gfap_aldh'], 
                        palette=['green','blue','magenta','gainsboro'],
                        save="_astrocytes_gfap_aldh_hfd15.svg", size=30, color_map=mymap, use_raw=False)

### Differential gene expression - hfd_15

In [117]:
astro_15 = adata_astro[:,~adata_astro.var.index.isin(['Gfap','Aldh1l1'])]

In [118]:
if bool_plot==True:
    astro15 = astro_15[(astro_15.obs['diet']=='hfd_15') & 
                    (astro_15.obs['gfap_aldh'].isin(['aldh_only','gfap_only'])), ]
    dets_mark = de.test.t_test(data=astro15, sample_description=astro15.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    dets_mark_summary
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50,
                           log2_fc_threshold=3.5, save=sc_settings_figdir+"dge_volcano_hfd15_aldh-gfap",
                           suffix="_astrocytes.svg")

In [119]:
if bool_plot==True:
    astro15 = astro_15[(astro_15.obs['diet']=='hfd_15') & 
                    (astro_15.obs['gfap_aldh'].isin(['both','gfap_only'])), ]
    dets_mark = de.test.t_test(data=astro15, sample_description=astro15.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50, 
                           log2_fc_threshold=3.5, save=sc_settings_figdir+"dge_volcano_hfd15_both-gfap",
                           suffix="_astrocytes.svg")

In [120]:
if bool_plot==True:
    astro15 = astro_15[(astro_15.obs['diet']=='hfd_15') & 
                    (astro_15.obs['gfap_aldh'].isin(['aldh_only','both'])), ]
    dets_mark = de.test.t_test(data=astro15, sample_description=astro15.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50, 
                           log2_fc_threshold=3.5, save=sc_settings_figdir+"dge_volcano_hfd15_aldh-both",
                           suffix="_astrocytes.svg")

# Define Cell Types

<a id="DE"></a>

## DE Genes

In [121]:
sc.tl.rank_genes_groups(adata_astro, groupby='louvain', key_added='rank_genes')

if bool_plot==True:
    sc.pl.rank_genes_groups(adata_astro, key='rank_genes', groups=['0','1','2'], save="_astorcytes_1.svg")
    sc.pl.rank_genes_groups(adata_astro, key='rank_genes', groups=['3','4'], save="_astorcytes_2.svg")


ranking genes
... storing 'gfap_aldh' as categorical
    finished: added to `.uns['rank_genes']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)


## Summary heatmap, dotplot and stacked_violin for cluster assignments

In [122]:
if bool_plot==True:
    sc.pl.heatmap(
        adata=adata_astro, 
        var_names=marker_genes_dict, 
        groupby="louvain", 
        use_raw=False, 
        log=False, 
        dendrogram=True, 
        var_group_rotation=90, 
        show_gene_labels=True, 
        show=True, 
        save="_astrocytes_celltypes.svg"
    )

In [123]:
if bool_plot==True:
    sc.pl.dotplot(
        adata=adata_astro,
        var_names=marker_genes_dict, 
        groupby='louvain',
        use_raw=False, 
        log=False, 
        dendrogram=True, 
        var_group_rotation=90, 
        show=True, 
        save="_astrocytes_celltypes.svg")

In [124]:
if bool_plot==True:
    sc.pl.stacked_violin(
        adata=adata_astro, 
        var_names=marker_genes_dict, 
        groupby='louvain', 
        use_raw=False,
        dendrogram=True,
        cmap='viridis_r',
        show=True,
        save="_astrocytes_celltypes.svg")

In [125]:
if bool_plot==True:
    plt.figure(figsize=(7,7))
    cell_annotation = sc.tl.marker_gene_overlap(adata_astro, marker_genes_dict, key='rank_genes', 
                                                normalize='data')
    sb.heatmap(cell_annotation, cbar=False, annot=True)
    plt.savefig(sc_settings_figdir+'heatmap_astrocytes_rank_genes_cell_annotation.svg')

In [126]:
if bool_plot==True:
    sc.tl.embedding_density(adata_astro, basis='umap', groupby='diet')
    sc.pl.embedding_density(adata_astro, basis='umap', key='umap_density_diet',
                            group=['chow', 'hfd_5', 'hfd_15'], bg_dotsize=5, fg_dotsize=30, save="astrocytes.svg")

# Cluster 0,1, 2 only

<a id="Embedding"></a>

## Embedding and Clustering

In [127]:
if bool_recomp == True:  
    cell_ids_012 = np.asarray(adata_proc.obs_names)[
        [x in ['astrocytes','ependymal'] 
         for x in np.asarray(adata_proc.obs['celltypes'].values)]
    ]
    adata_012 = adata_raw[cell_ids_012,:].copy()
    adata_012.obs['n_genes'] = (adata_012.X > 0).sum(1)
    adata_012.obs['n_counts'] = adata_012.X.sum(1)
    mt_gene_mask = [gene.startswith('mt-') for gene in adata_012.var_names]
    temp_mt_sum = adata_012[:,mt_gene_mask].X.sum(1)
    temp_mt_sum = np.squeeze(np.asarray(temp_mt_sum))
    temp_n_counts = adata_012.obs['n_counts']
    adata_012.obs['mt_frac'] = temp_mt_sum/adata_012.obs['n_counts']
    adata_012.raw = adata_012
    sc.pp.normalize_per_cell(adata_012)
    sc.pp.log1p(adata_012)
    sc.pp.highly_variable_genes(adata_012,n_top_genes=4000)
    sc.pl.highly_variable_genes(adata_012)
    adata_012.X = adata_012.X.toarray()
    
    sc.pp.pca(adata_012, n_comps=50, use_highly_variable = True, random_state=0, svd_solver='arpack')
    sc.pp.neighbors(adata_012, n_neighbors=100, knn=True, method='umap', n_pcs=50, random_state=0)
    sc.tl.umap(adata_012)
    if bool_recluster == True:
        sc.tl.louvain(adata_012, resolution=1, flavor='vtraag', random_state=0)
        pd.DataFrame(adata_012.obs).to_csv(path_or_buf=sc_settings_writedir+'obs_adata_012.csv')
    else:
        obs = pd.read_csv(sc_settings_writedir+'obs_adata_012.csv')
        adata_012.obs['louvain']=pd.Series(obs['louvain'].values, dtype = 'category')
    sc.write(sc_settings_writedir+'adata_012.h5ad',adata_012)
else:
    adata_012 = sc.read(sc_settings_writedir+'adata_012.h5ad') 
sc.tl.paga(adata_012)

running PAGA
    finished: added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns) (0:00:00)


In [128]:
if bool_plot == True:
    cf.plot_umap_marker(adata_012, ['louvain'], save="_clust012_louvain.svg", use_raw=False, size=5)

In [129]:
if bool_plot == True:
    cf.plot_umap_marker(adata_012, ['louvain'], save="_clust012_louvain_ondata.svg", use_raw=False, legend_loc='on data', size=5)

In [130]:
if bool_plot == True:
    cf.plot_umap_marker(adata_012, ['n_genes','n_counts','mt_frac'], color_map=mymap, size=10, save="_clust012_n_gene_count_mt.svg", use_raw = False)

A high fraction of mitochondrial RNA is in cluster 3 and around the central cluster 8.

In [131]:
if bool_plot == True:
    cf.plot_umap_marker(adata_012, ['Gfap','Aldh1l1'], color_map=mymap, size=10, save="_clust012_markers.svg", use_raw = False)

In [132]:
if bool_plot == True:
    cf.plot_umap_marker(adata_012, ['diet'], save="_clust012_diet.svg", size=10, use_raw=False)

In [133]:
if bool_plot == True:
    sc.pl.paga(adata_012, save="_clust012.svg")

In [134]:
if bool_plot == True:
    cf.cell_percent(adata_012, cluster='louvain', condition='diet', xlabel='clusters', ylabel='percentage', 
                 title='barplot_clust012_diet_per_clusters', save=sc_settings_figdir, table=False)

In [135]:
if bool_plot == True:
    aldh_pos = adata_012.obs_names[np.asarray(adata_012[:,'Aldh1l1'].X).flatten()>0]
    gfap_pos = adata_012.obs_names[np.asarray(adata_012[:,'Gfap'].X).flatten()>0]
    glast_pos = adata_012.obs_names[np.asarray(adata_012[:,'Slc1a3'].X).flatten()>0]

    matplotlib_venn.venn3([
        set(aldh_pos),
        set(gfap_pos),
        set(glast_pos)
    ], set_labels = ("Aldh1l1", "Gfap", "Slc1a3"))
    plt.savefig(sc_settings_figdir+'venndiagram_clust012_gfap-aldh-glast.svg')

In [136]:
if bool_plot == True:
    aldh_pos = adata_012.obs_names[np.asarray(adata_012[:,'Aldh1l1'].X).flatten()>0]
    gfap_pos = adata_012.obs_names[np.asarray(adata_012[:,'Gfap'].X).flatten()>0]

    matplotlib_venn.venn2([
        set(aldh_pos),
        set(gfap_pos)
    ], set_labels = ("Aldh1l1", "Gfap"), set_colors = ('lime','magenta'))
    plt.savefig(sc_settings_figdir+'venndiagram_clust012_gfap-aldh.svg')

### Count distribution for Aldh1l1 and Gfap

#### Single and double positive counts¶

Define booleans for single and double positive counts of Gfap and Aldh1l1

In [137]:
non_boolean_int = np.array((adata_012[:,'Gfap'].X<=0) & (adata_012[:,'Aldh1l1'].X<=0), dtype=int)

gfap_single_boolean = (adata_012[:,'Gfap'].X>0) & (adata_012[:,'Aldh1l1'].X<=0)
aldh_single_boolean = (adata_012[:,'Aldh1l1'].X>0) & (adata_012[:,'Gfap'].X<=0) 
single_boolean_int = np.array((gfap_single_boolean | aldh_single_boolean), dtype=int)*1

gfap_aldh_double_boolean = (adata_012[:,'Gfap'].X>0) & (adata_012[:,'Aldh1l1'].X>0)
double_boolean_int = np.array((gfap_aldh_double_boolean), dtype=int)*2

non_boolean_int *=0

In [138]:
gfap_single_pos = adata_012[:,'Gfap'].X[gfap_single_boolean]
aldh_single_pos = adata_012[:,'Aldh1l1'].X[aldh_single_boolean]

print('Gfap Single Positive ',len(gfap_single_pos))
print('Aldh1l1 Single Positive ',len(aldh_single_pos))

gfap_aldh_double_pos = adata_012[:,'Gfap'].X[gfap_aldh_double_boolean]

print('Gfap/Aldh1l1 Double Positive ',len(gfap_aldh_double_pos))


Gfap Single Positive  1703
Aldh1l1 Single Positive  964
Gfap/Aldh1l1 Double Positive  738


<a id="adipmarkers"></a>

In [139]:
adata_012.obs['sdt_pos'] = np.array((non_boolean_int + single_boolean_int + double_boolean_int), dtype=str)
adata_012.obs['s_pos'] = np.array((np.array(gfap_single_boolean, dtype=int)*1)+(np.array(aldh_single_boolean, dtype=int)*2), dtype=str)
adata_012.obs['d_pos'] = np.array((np.array(gfap_aldh_double_boolean, dtype=int)*1), dtype=str)

# make them categorical
adata_012.obs['sdt_pos'] = pd.Series(adata_012.obs['sdt_pos'], dtype="category")
adata_012.obs['s_pos'] = pd.Series(adata_012.obs['s_pos'], dtype="category")
adata_012.obs['d_pos'] = pd.Series(adata_012.obs['d_pos'], dtype="category")

In [140]:
if bool_plot == True:
    new_cluster_names = ['Non Positives','Single Positives','Double Positives']
    adata_012.rename_categories('sdt_pos', new_cluster_names)
    sc.pl.umap(adata_012, color=['sdt_pos'], size=10, palette=["lightgrey","tomato","blue"], 
               save="_clust012_gfap-aldh_single_double_positiv.svg")
    new_cluster_names = ['Non Single Positives','Gfap Single Positives','Aldh1l1 Single Positives']
    adata_012.rename_categories('s_pos', new_cluster_names)
    sc.pl.umap(adata_012[adata_012.obs['s_pos']!='Non Single Positives'], color=['s_pos'],
               size=10, palette=["magenta","lime"], save="_clust012_gfap-aldh_single_positiv.svg")
    new_cluster_names = ['Non Double Positives','Gfap/Aldh1l1 Double Positive']
    adata_012.rename_categories('d_pos', new_cluster_names)
    sc.pl.umap(adata_012[adata_012.obs['d_pos']!='Non Double Positives'], color=['d_pos'], size=10,
              save="_clust012_gfap-aldh_doble_positiv.svg")

### Cluster 0,1,2 chow diet

In [141]:
if bool_plot == True:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4), gridspec_kw={'wspace':0.8})
    ax1_dict = ax1.scatter(adata_012[adata_012.obs['diet']=='chow', ][:, ['Gfap']].X, 
                           adata_012[adata_012.obs['diet']=='chow', ][:, ['Aldh1l1']].X, 
                           s=9, cmap='seismic', c='dodgerblue')
    ax1.set_title('chow')
    ax1.set_ylabel('Gfap')
    ax1.set_xlabel('Aldh1l1')
    ax2_dict = sc.pl.umap(adata_012[adata_012.obs['diet']=='chow', ], color=['louvain'], size=10, ax=ax2, show=False)
    plt.savefig(sc_settings_figdir+'gfap-aldh_clust012_chow.svg')

In [142]:
if bool_plot == True:
    cf.plot_umap_marker(adata_012, ['Gfap', 'Aldh1l1'], save="_clust012_chow_gfap-aldh.svg", size=10, 
                        color_map=mymap, use_raw=False)

In [143]:
if bool_plot == True:
    aldh_pos = adata_012[adata_012.obs['diet']=='chow', ].obs_names[np.asarray(adata_012[adata_012.obs['diet']=='chow', ][:,'Aldh1l1'].X).flatten()>0]
    gfap_pos = adata_012[adata_012.obs['diet']=='chow', ].obs_names[np.asarray(adata_012[adata_012.obs['diet']=='chow', ][:,'Gfap'].X).flatten()>0]

    matplotlib_venn.venn2([
        set(aldh_pos),
        set(gfap_pos)
    ], set_labels = ("Aldh1l1", "Gfap"), set_colors = ('lime','magenta'))
    plt.savefig(sc_settings_figdir+'venndiagram_clust012_chow_gfap-aldh.svg')

### Cluster 0,1,2 hfd_5 diet

In [144]:
if bool_plot == True:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4), gridspec_kw={'wspace':0.8})
    ax1_dict = ax1.scatter(adata_012[adata_012.obs['diet']=='hfd_5', ][:, ['Gfap']].X, 
                           adata_012[adata_012.obs['diet']=='hfd_5', ][:, ['Aldh1l1']].X, 
                           s=9, cmap='seismic', c='darkorange')
    ax1.set_title('hfd_5')
    ax1.set_ylabel('Gfap')
    ax1.set_xlabel('Aldh1l1')
    ax2_dict = sc.pl.umap(adata_012[adata_012.obs['diet']=='hfd_5', ], color=['louvain'], size=10, ax=ax2, show=False)
    plt.savefig(sc_settings_figdir+'gfap-aldh_clust012_hfd5.svg')

In [145]:
if bool_plot == True:
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='hfd_5', ], ['Gfap', 'Aldh1l1'],
                        save="_clust012_hfd5_gfap-aldh.svg", size=10, color_map=mymap, use_raw=False)

In [146]:
if bool_plot == True:
    aldh_pos = adata_012[adata_012.obs['diet']=='hfd_5', ].obs_names[np.asarray(adata_012[adata_012.obs['diet']=='hfd_5', ][:,'Aldh1l1'].X).flatten()>0]
    gfap_pos = adata_012[adata_012.obs['diet']=='hfd_5', ].obs_names[np.asarray(adata_012[adata_012.obs['diet']=='hfd_5', ][:,'Gfap'].X).flatten()>0]

    matplotlib_venn.venn2([
        set(aldh_pos),
        set(gfap_pos)
    ], set_labels = ("Aldh1l1", "Gfap"), set_colors = ('lime','magenta'))
    plt.savefig(sc_settings_figdir+'venndiagram_clust012_hfd5_gfap-aldh.svg')

### Cluster 0,1,2 hfd_15 diet

In [147]:
if bool_plot == True:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4), gridspec_kw={'wspace':0.8})
    ax1_dict = ax1.scatter(adata_012[adata_012.obs['diet']=='hfd_15', ][:, ['Gfap']].X, 
                           adata_012[adata_012.obs['diet']=='hfd_15', ][:, ['Aldh1l1']].X, 
                           s=9, cmap='seismic', c='green')
    ax1.set_title('hfd_15')
    ax1.set_ylabel('Gfap')
    ax1.set_xlabel('Aldh1l1')
    ax2_dict = sc.pl.umap(adata_012[adata_012.obs['diet']=='hfd_15', ], color=['louvain'], size=10, ax=ax2, show=False)
    plt.savefig(sc_settings_figdir+'gfap-aldh_clust012_hfd15.svg')

In [148]:
if bool_plot == True:
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='hfd_15', ], ['Gfap', 'Aldh1l1'],
                        save="_clust012_hfd15_gfap-aldh.svg", size=10, color_map=mymap, use_raw=False)

In [149]:
if bool_plot == True:
    aldh_pos = adata_012[adata_012.obs['diet']=='hfd_15', ].obs_names[np.asarray(adata_012[adata_012.obs['diet']=='hfd_15', ][:,'Aldh1l1'].X).flatten()>0]
    gfap_pos = adata_012[adata_012.obs['diet']=='hfd_15', ].obs_names[np.asarray(adata_012[adata_012.obs['diet']=='hfd_15', ][:,'Gfap'].X).flatten()>0]

    matplotlib_venn.venn2([
        set(aldh_pos),
        set(gfap_pos)
    ], set_labels = ("Aldh1l1", "Gfap"), set_colors = ('lime','magenta')) 
    plt.savefig(sc_settings_figdir+'venndiagram_clust012_hfd15_gfap-aldh.svg')

## Cells expression Gfap and Aldh1l1

In [150]:
adata_012.obs['gfap_aldh'] = np.select([((adata_012[:,'Gfap'].X>0) & (adata_012[:,'Aldh1l1'].X==0)), 
                                          ((adata_012[:,'Gfap'].X==0) & (adata_012[:,'Aldh1l1'].X>0)),
                                          ((adata_012[:,'Gfap'].X>0) & (adata_012[:,'Aldh1l1'].X>0)), 
                                          ((adata_012[:,'Gfap'].X==0) & (adata_012[:,'Aldh1l1'].X==0))],
                                          ['gfap_only','aldh_only','both','none'])

### Chow

In [151]:
if bool_plot==True:
    data = adata_012[adata_012.obs['diet']=='chow', ].obs['gfap_aldh'].value_counts().sort_index()
    def func(pct, allvals):
        absolute = int(pct/100.*np.sum(allvals))
        return "{:.1f}%\n({:d})".format(pct, absolute)

    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(16, 6), subplot_kw=dict(aspect="equal"))
    ax0 = sc.pl.umap(adata_012[adata_012.obs['diet']=='chow', ], 
                     color=['gfap_aldh'], size=10, ax=ax0, show=False)
    wedges, texts, autotexts = ax1.pie(data, autopct=lambda pct: func(pct, data), textprops=dict(color="w"))
    ax1.set_title('Chow diet cells - markers share')

    fig.savefig(sc_settings_figdir+'umap_pie_chart_clust012_chow_gfap-aldh.svg')

### HFD_5

In [152]:
if bool_plot==True:
    data = adata_012[adata_012.obs['diet']=='hfd_5', ].obs['gfap_aldh'].value_counts().sort_index()
    def func(pct, allvals):
        absolute = int(pct/100.*np.sum(allvals))
        return "{:.1f}%\n({:d})".format(pct, absolute)

    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(16, 6), subplot_kw=dict(aspect="equal"))
    ax0 = sc.pl.umap(adata_012[adata_012.obs['diet']=='hfd_5', ], 
                     color=['gfap_aldh'], size=10, ax=ax0, show=False)
    wedges, texts, autotexts = ax1.pie(data, autopct=lambda pct: func(pct, data), textprops=dict(color="w"))
    ax1.set_title('Chow diet cells - markers share')

    fig.savefig(sc_settings_figdir+'umap_pie_chart_clust012_hfd5_gfap-aldh.svg')

### HFD_15

In [153]:
if bool_plot==True: 
    data = adata_012[adata_012.obs['diet']=='hfd_15', ].obs['gfap_aldh'].value_counts().sort_index()
    def func(pct, allvals):
        absolute = int(pct/100.*np.sum(allvals))
        return "{:.1f}%\n({:d})".format(pct, absolute)

    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(16, 6), subplot_kw=dict(aspect="equal"))
    ax0 = sc.pl.umap(adata_012[adata_012.obs['diet']=='hfd_15', ], 
                     color=['gfap_aldh'], size=10, ax=ax0, show=False)
    wedges, texts, autotexts = ax1.pie(data, autopct=lambda pct: func(pct, data), textprops=dict(color="w"))
    ax1.set_title('Chow diet cells - markers share')

    fig.savefig(sc_settings_figdir+'umap_pie_chart_clust012_hfd15_gfap-aldh.svg')

In [154]:
if bool_plot == True:
    cf.cell_percent(adata_012, cluster='diet', condition='gfap_aldh', xlabel='clusters', ylabel='percentage', 
                 title='barplot_per_diet_clust012_gfap-aldh', save=sc_settings_figdir, table=False)

## Per marker and diet

In [155]:
adata_012.obs['gfap_only'] = adata_012.obs['gfap_aldh']=='gfap_only'
adata_012.obs['aldh_only'] = adata_012.obs['gfap_aldh']=='aldh_only'
adata_012.obs['both'] = adata_012.obs['gfap_aldh']=='both'

In [156]:
# make them categorical
adata_012.obs['gfap_only'] = pd.Series(adata_012.obs['gfap_only'], dtype="category")
adata_012.obs['aldh_only'] = pd.Series(adata_012.obs['aldh_only'], dtype="category")
adata_012.obs['both'] = pd.Series(adata_012.obs['both'], dtype="category")

In [157]:
adata_012.rename_categories('gfap_only', ['none','gfap_only'])

In [158]:
adata_012.rename_categories('aldh_only', ['none','aldh_only'])
adata_012.rename_categories('both', ['none','both'])

### chow

In [159]:
if bool_plot==True:
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='chow', ], ['gfap_only'], palette=['gainsboro','magenta'],
                        save="_clust012_gfap_only_chow.svg", size=10, color_map=mymap, use_raw=False)

In [160]:
if bool_plot==True:
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='chow', ], ['aldh_only'], palette=['gainsboro','green'],
                        save="_clust012_aldh_only_chow.svg", size=10, color_map=mymap, use_raw=False)

In [161]:
if bool_plot==True:
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='chow', ], ['both'], palette=['gainsboro','blue'],
                        save="_clust012_both_chow.svg", size=10, color_map=mymap, use_raw=False)

In [162]:
if bool_plot==True: 
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='chow', ], ['gfap_aldh'], 
                        palette=['green','blue','magenta','gainsboro'],
                        save="_clust012_gfap_aldh_chow.svg", size=10, color_map=mymap, use_raw=False)

### Differential gene expression - chow

In [163]:
astro_0 = adata_012[:,~adata_012.var.index.isin(['Gfap','Aldh1l1'])]

In [164]:
if bool_plot==True:
    astro0 = astro_0[(astro_0.obs['diet']=='chow') & 
                    (astro_0.obs['gfap_aldh'].isin(['aldh_only','gfap_only'])), ]
    dets_mark = de.test.t_test(data=astro0, sample_description=astro0.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    dets_mark_summary
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05,corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50, 
                           log2_fc_threshold=6, save=sc_settings_figdir+"dge_volcano_chow_aldh-gfap",
                           suffix="_clust012.svg")

In [165]:
if bool_plot==True:
    astro0 = astro_0[(astro_0.obs['diet']=='chow') & 
                    (astro_0.obs['gfap_aldh'].isin(['both','gfap_only'])), ]
    dets_mark = de.test.t_test(data=astro0, sample_description=astro0.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05,corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50,
                           log2_fc_threshold=6, save=sc_settings_figdir+"dge_volcano_chow_both-gfap",
                           suffix="_clust012.svg")

In [166]:
if bool_plot==True:
    astro0 = astro_0[(astro_0.obs['diet']=='chow') & 
                    (astro_0.obs['gfap_aldh'].isin(['aldh_only','both'])), ]
    dets_mark = de.test.t_test(data=astro0, sample_description=astro0.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50, 
                           log2_fc_threshold=6, save=sc_settings_figdir+"dge_volcano_chow_aldh-both",
                           suffix="_clust012.svg")

### hfd 5

In [167]:
if bool_plot==True:
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='hfd_5', ], ['gfap_only'], palette=['gainsboro','magenta'],
                        save="_clust012_gfap_only_hfd5.svg", size=10, color_map=mymap, use_raw=False)

In [168]:
if bool_plot==True:
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='hfd_5', ], ['aldh_only'], palette=['gainsboro','green'],
                        save="_clust012_aldh_only_hfd5.svg", size=10, color_map=mymap, use_raw=False)

In [169]:
if bool_plot==True:
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='hfd_5', ], ['both'], palette=['gainsboro','blue'],
                        save="_clust012_both_hfd5.svg", size=10, color_map=mymap, use_raw=False)

In [170]:
if bool_plot==True:
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='hfd_5', ], ['gfap_aldh'],
                        palette=['green','blue','magenta','gainsboro'],
                        save="_clust012_gfap_aldh_hfd5.svg", size=10, color_map=mymap, use_raw=False)

### Differential gene expression - hfd_5

In [171]:
astro_5 = adata_012[:,~adata_012.var.index.isin(['Gfap','Aldh1l1'])]

In [172]:
if bool_plot==True:
    astro5 = astro_5[(astro_5.obs['diet']=='hfd_5') & 
                    (astro_5.obs['gfap_aldh'].isin(['aldh_only','gfap_only'])), ]
    dets_mark = de.test.t_test(data=astro5, sample_description=astro5.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    dets_mark_summary
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50, 
                           log2_fc_threshold=6, save=sc_settings_figdir+"dge_volcano_hfd5_aldh-gfap",
                           suffix="_clust012.svg")

In [173]:
if bool_plot==True:
    astro5 = astro_5[(astro_5.obs['diet']=='hfd_5') & 
                    (astro_5.obs['gfap_aldh'].isin(['both','gfap_only'])), ]
    dets_mark = de.test.t_test(data=astro5, sample_description=astro5.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50,
                           log2_fc_threshold=6, save=sc_settings_figdir+"dge_volcano_hfd5_both-gfap",
                           suffix="_clust012.svg")

In [174]:
if bool_plot==True:
    astro5 = astro_5[(astro_5.obs['diet']=='hfd_5') & 
                    (astro_5.obs['gfap_aldh'].isin(['aldh_only','both'])), ]
    dets_mark = de.test.t_test(data=astro5, sample_description=astro5.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50, 
                           log2_fc_threshold=6, save=sc_settings_figdir+"dge_volcano_hfd5_aldh-both",
                           suffix="_clust012.svg")

### hfd 15

In [175]:
if bool_plot==True:
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='hfd_15', ], ['gfap_only'], palette=['gainsboro','magenta'],
                        save="_clust012_gfap_only_hfd15.svg", size=10, color_map=mymap, use_raw=False)

In [176]:
if bool_plot==True:
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='hfd_15', ], ['aldh_only'], palette=['gainsboro','green'],
                        save="_clust012_aldh_only_hfd15.svg", size=10, color_map=mymap, use_raw=False)

In [177]:
if bool_plot==True:
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='hfd_15', ], ['both'], palette=['gainsboro','blue'],
                        save="_clust012_both_hfd15.svg", size=10, color_map=mymap, use_raw=False)

In [178]:
if bool_plot==True:
    cf.plot_umap_marker(adata_012[adata_012.obs['diet']=='hfd_15', ], ['gfap_aldh'], 
                        palette=['green','blue','magenta','gainsboro'],
                        save="_clust012_gfap_aldh_hfd15.svg", size=10, color_map=mymap, use_raw=False)

### Differential gene expression - hfd_15

In [179]:
astro_15 = adata_012[:,~adata_012.var.index.isin(['Gfap','Aldh1l1'])]

In [180]:
if bool_plot==True:
    astro15 = astro_15[(astro_15.obs['diet']=='hfd_15') & 
                    (astro_15.obs['gfap_aldh'].isin(['aldh_only','gfap_only'])), ]
    dets_mark = de.test.t_test(data=astro15, sample_description=astro15.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    dets_mark_summary
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50,
                           log2_fc_threshold=6, save=sc_settings_figdir+"dge_volcano_hfd15_clust012_aldh-gfap")

In [181]:
if bool_plot==True:
    astro15 = astro_15[(astro_15.obs['diet']=='hfd_15') & 
                    (astro_15.obs['gfap_aldh'].isin(['both','gfap_only'])), ]
    dets_mark = de.test.t_test(data=astro15, sample_description=astro15.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50, 
                           log2_fc_threshold=6, save=sc_settings_figdir+"dge_volcano_hfd15_clust012_both-gfap")

In [182]:
if bool_plot==True:
    astro15 = astro_15[(astro_15.obs['diet']=='hfd_15') & 
                    (astro_15.obs['gfap_aldh'].isin(['aldh_only','both'])), ]
    dets_mark = de.test.t_test(data=astro15, sample_description=astro15.obs, grouping="gfap_aldh", is_logged=False)
    dets_mark_summary = dets_mark.summary(mean_thres=np.log(0.01), qval_thres=0.1, fc_lower_thres=1, fc_upper_thres=1)
    #dets_age_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_age.tab", sep="\t")
    dets_mark.plot_volcano(alpha=0.05, corrected_pval=False, min_fc=1, size=15, log10_p_threshold=-50, 
                           log2_fc_threshold=5.6, save=sc_settings_figdir+"dge_volcano_hfd15_clust012_aldh-both")

# Define Cell Types

<a id="DE"></a>

## DE Genes

In [183]:
sc.tl.rank_genes_groups(adata_012, groupby='louvain', key_added='rank_genes')

if bool_plot==True:
    sc.pl.rank_genes_groups(adata_012, key='rank_genes', groups=['0','1','2'], save="_clust012_1.svg")
    sc.pl.rank_genes_groups(adata_012, key='rank_genes', groups=['3','4','5'], save="_clust012_2.svg")
    sc.pl.rank_genes_groups(adata_012, key='rank_genes', groups=['6','7'], save="_clust012_3.svg")

ranking genes
... storing 'gfap_aldh' as categorical
    finished: added to `.uns['rank_genes']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:01)


## Summary heatmap, dotplot and stacked_violin for cluster assignments

In [184]:
if bool_plot==True:
    sc.pl.heatmap(
        adata=adata_012, 
        var_names=marker_genes_dict, 
        groupby="louvain", 
        use_raw=False, 
        log=False, 
        dendrogram=True, 
        var_group_rotation=90, 
        show_gene_labels=True, 
        show=True, 
        save="_clust012_celltypes.svg"
    )

In [185]:
if bool_plot==True:
    sc.pl.dotplot(
        adata=adata_012,
        var_names=marker_genes_dict, 
        groupby='louvain',
        use_raw=False, 
        log=False, 
        dendrogram=True, 
        var_group_rotation=90, 
        show=True, 
        save="_clust012_celltypes.svg")

In [186]:
if bool_plot==True:
    sc.pl.stacked_violin(
        adata=adata_012, 
        var_names=marker_genes_dict, 
        groupby='louvain', 
        use_raw=False,
        dendrogram=True,
        cmap='viridis_r',
        show=True,
        save="_clust012_celltypes.svg")

In [187]:
if bool_plot==True:
    plt.figure(figsize=(7,7))
    cell_annotation = sc.tl.marker_gene_overlap(adata_012, marker_genes_dict, key='rank_genes', 
                                                normalize='data')
    sb.heatmap(cell_annotation, cbar=False, annot=True)
    plt.savefig(sc_settings_figdir+'heatmap_clust012_rank_genes_cell_annotation.svg')

In [188]:
if bool_plot==True:
    sc.tl.embedding_density(adata_012, basis='umap', groupby='diet')
    sc.pl.embedding_density(adata_012, basis='umap', key='umap_density_diet',
                            group=['chow', 'hfd_5', 'hfd_15'], bg_dotsize=5, fg_dotsize=30, save="clust012.svg")